In [8]:
import grobid_tei_xml
import pandas as pd
import os

In [21]:
def parse_grobid_xml(file_path):
    with open(file_path, "r", encoding="utf-8") as xml_file:
        doc = grobid_tei_xml.parse_document_xml(xml_file.read())
        title = "Title: " + doc.header.title if doc.header.title else ""
        abstract = doc.abstract or ""
        body = doc.body or ""
        index = file_path.split("/")[-1].split(".")[0]
        return f"Paper #: {index}\n{title}\n{abstract}\n{body}\n" #title, abstract, body

In [ ]:
def convert_grobid_xml_to_csv(input_dir, output_file):
    crossref_df = pd.read_csv("data/crossref_data.csv")
    crossref_df["text"] = None
    for index, row in crossref_df.iterrows():
        i = 1
        added_count = 0
        paper_text = ""
        while added_count < 3 and i <= 6: #prevents more than 3 files from being added, up to 6 accounts for grobid failures to generate xml
            file_path = f"{input_dir}/{index}_{i}.xml"
            if os.path.exists(file_path):
                paper_text += parse_grobid_xml(file_path)
                added_count += 1
            i += 1
        crossref_df.at[index, "text"] = paper_text
    output_df = crossref_df[crossref_df["text"].str.len() > 0]
    unique_dois = output_df.groupby('DOI').first()
    unique_dois.to_csv(output_file)
    return unique_dois

In [ ]:
output_df = convert_grobid_xml_to_csv("data/xmls", "data/scraped_papers.csv")